# GeoLlama: Fine-Tuning on Topoym Extraction
This notebook wlaks through the process of producing the `GeoLlama_toponym` model used to extract toponyms form text. We will be using the LGL and GeoVirus datasets to fine-tune the model. These have been pre-processed and combined into the `llama3_toponym_extraction_ft.json` dataset.

We will use Unsloth to optimize the fine tuning process. Thi will make for fater, more memory efficient training, alhtough it does mean the training can only be done on a Linux machine. A Google Colab GPU instance is optimal for this.

Fine tuning require ~20GB of GPU RAM, depending on GPU model.

## 1. Setting up the Unsloth package

In [31]:
# standard library imports
import random
import json
import os
# third party imports
from google.colab import drive
from unsloth import FastLanguageModel, is_bfloat16_supported
from trl import SFTTrainer
from transformers import TrainingArguments
from huggingface_hub import login
from datasets import Dataset

import pandas as pd
import numpy as np
import torch

#drive.mount('/content/drive').

In [3]:
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
max_seq_length = 2048
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)



config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.7
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.25.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # 4x longer contexts auto supported!
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)



Unsloth 2024.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## 2. Data Pre-processing
We will use the `llama3_toponym_extraction_ft.json` dataset to fine-tune and test the model. The dataset has 817 examples, which isn't a huge number, but the relative simplicty of the task and the requirement for a highly structured output means we don't need an extremely large training set.

We'll use 750 samples from the dataset as training data and use the remaining 67 samples to test the model.

In [6]:
# load the data
with open('../data/fine_tuning_data/llama3_toponym_extraction_ft.json') as f:
    data = json.load(f)

In [7]:
# Set up the prompt template for model training

RAG_prompt = """Below is an instruction that describes a task, paired with an input that provides a specfic example which the task should be applied to. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}
"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["response"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = RAG_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

ft_data = {"items":data}

In [10]:
# Set up the test/train split
random.seed(7723)
trn_idxs = random.sample(range(len(data)), 750)
val_idxs = [x for x in range(len(data)) if x not in trn_idxs]
trn_data = [data[i] for i in trn_idxs]
val_data = [data[i] for i in val_idxs]

trn_dataset = Dataset.from_pandas(pd.DataFrame(trn_data), split="train")

In [ ]:
# format data for input into the model
trn_dataset = trn_dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/3600 [00:00<?, ? examples/s]

## 3. Training
This section shows the process of model training. Training takes ~45 minutes on a T4 GPU. We will use the `SFTTrainer` class to train the model, setting the parameters as shown below.

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = trn_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        num_train_epochs = 1,
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Map (num_proc=2):   0%|          | 0/3600 [00:00<?, ? examples/s]

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA L4. Max memory = 22.168 GB.
5.594 GB of memory reserved.


In [ ]:
# trian the model
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 3,600 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 450
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.959900
2,1.775900
3,1.748300
4,1.833800
5,1.669700
6,1.629300
7,1.706500
8,1.500900
9,1.460000
10,1.419100


In [ ]:

#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")



2730.1956 seconds used for training.
45.5 minutes used for training.
Peak reserved memory = 8.967 GB.
Peak reserved memory for training = 3.373 GB.
Peak reserved memory % of max memory = 40.45 %.
Peak reserved memory for training % of max memory = 15.216 %.


In [ ]:
# log in to huggingface so the model can be saved
login()

In [ ]:
# save to huggingface
model.push_to_hub("JoeShingleton/GeoLlama-3.1-8b-toponym")

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Saved model to https://huggingface.co/JoeShingleton/GeoLlama


## 4. Testing the model
We will now test the model with our 67 testing samples. We'll use the `TopoModel` class form the `geo_llama` package as a wrapper for the fine-tuned model. This module will handle cleaning up any responses and processing inputs.

We'll assess the accuracy as the proportion of unique toponyms form the text which have been found by the model. Note: we are not expecting the model to identify the location of the toponym within the text, so the overlap will not be assessed.

In [21]:
from geo_llama.model import TopoModel

# set up the model
model = TopoModel(model_name='JoeShingleton/GeoLlama-3.1-8b-toponym',
                  prompt_path='../data/prompt_templates/prompt_template.txt',
                  instruct_path='../data/prompt_templates/topo_instruction.txt',
                  input_path=None,
                  config_path='../data/config_files/model_config.json')

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.25.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [53]:
from tqdm import tqdm

outputs = []
for d in tqdm(val_data):
  prompt = model.toponym_prompt(d['input'])
  tst_outputs = model.get_output(prompt['text'], d['input'])
  outputs.append(tst_outputs)


  1%|▏         | 1/67 [00:02<02:18,  2.10s/it]


{'toponyms': ['Pointe Coupee', 'St. James']}



  3%|▎         | 2/67 [00:08<04:51,  4.49s/it]


{'toponyms': ['Minnesota', 'Fargo', 'Moorhead', 'Nebraska', 'Fargo-Moorhead', 'Red River Valley', 'Oakport', 'North Dakota', 'Alexandria']}



  4%|▍         | 3/67 [00:12<04:40,  4.39s/it]

...


{'toponyms': ['São Paulo', 'Japan', 'U.S.', 'Minas Gerais', 'New South Wales', 'Spain', 'Toronto', 'New York', 'Tocantins', 'U.S. Centers for Disease Control and Prevention', 'Ho Chi Minh City', 'Victoria', 'Moscow', 'Glasgow', 'Santa Catarina', 'Paisley', 'Istanbul', 'United Kingdom', 'Santiago', 'Canad', 'China', 'England', 'Lebanon', 'United States', 'New Zealand', 'Vietnam', 'Nicaragua', 'American', 'Australia', 'Russia', 'Philippines', 'Iraq', 'Iran', 'Turkey', 'Egypt', 'Bulgaria', 'Scotland', 'Brazil', 'Canada', 'Singapore', 'Vietname', 'US', 'Pensylvania', 'Vietnams', 'California', 'U.S.', 'Australia', 'Vietnam', 'Lebanon', 'U.S.']}



 99%|█████████▊| 66/67 [04:17<00:09,  9.19s/it]


{'toponyms': ['New Zealand', 'Tennessee', 'Delaware', 'Kentucky', 'Cairo', 'Illinois', 'Hong Kong', 'China', 'Kentucky', 'Alabama', 'Nigeria', 'Shanghai', 'New Brunswick', 'United States', 'South Carolina', 'Mexico', 'Fort Worth', 'US', 'British Columbia', 'Houston', 'Texas', 'African', 'Bavaria', 'Canada', 'Egypt', 'Spain', 'German', 'American', 'Alberta', 'North America']}



100%|██████████| 67/67 [04:19<00:00,  3.88s/it]


{'toponyms': ['San Jose', 'Santa Clara County', 'Monterey Highway', 'California', 'Las Vegas', 'Monterey']}



In [55]:
# Get the precision, recall and F1 on each article.
topo_precision = []
topo_recall = []
topo_f1 = []
topo_distance = []

def get_toponym_metrics(true_toponyms:list, pred_toponyms:list)->dict[str,int]:
    true_positives = len([t for t in pred_toponyms if t in true_toponyms])
    false_positives = len([t for t in pred_toponyms if t not in true_toponyms])
    false_negatives = len([t for t in true_toponyms if t not in pred_toponyms])
    return {'TP':true_positives, 'FP':false_positives, 'FN':false_negatives}

def get_accuracy_metrics(topo_metrics: dict[str, int]) -> dict[str, float]:
    tp = topo_metrics['TP']
    fp = topo_metrics['FP']
    fn = topo_metrics['FN']

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0

    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    return {'precision': precision, 'recall': recall, 'F1': f1}

for pred, true in zip(outputs, [x['response'] for x in val_data]):
    pred_toponyms = pred['toponyms']
    true_toponyms = json.loads(true.replace("'", '"'))['toponyms']
   # get true/false positives/negatives
    toponym_metrics = get_toponym_metrics(true_toponyms, pred_toponyms)
  # get prec, rec, f1
    accuracy_metrics = get_accuracy_metrics(toponym_metrics)
    topo_precision.append(accuracy_metrics['precision'])
    topo_recall.append(accuracy_metrics['recall'])
    topo_f1.append(accuracy_metrics['F1'])

In [56]:
# print the results
macro_precision = np.mean(topo_precision)
macro_recall = np.mean(topo_recall)
macro_f1 = np.mean(topo_f1)

print(f'Macro precision: {macro_precision:.3f}')
print(f'Macro recall: {macro_recall:.3f}')
print(f'Macro F1: {macro_f1:.3f}')

Macro precision: 0.892
Macro recall: 0.859
Macro F1: 0.863
